In [1]:
import os
import torch

import csv 

import pandas as pd
import numpy as np

In [2]:
disease_names = np.array(['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 
                 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
                'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'No Finding'])

In [3]:
data = []
names = []
target = []

data_sheet = pd.read_csv("/home/user/Chest_X_Ray_Data/CXR14/CXR8/Data_Entry_2017_v2020.csv")

print(data_sheet)

N = data_sheet.shape[0]

             Image Index          Finding Labels  Follow-up #  Patient ID  \
0       00000001_000.png            Cardiomegaly            0           1   
1       00000001_001.png  Cardiomegaly|Emphysema            1           1   
2       00000001_002.png   Cardiomegaly|Effusion            2           1   
3       00000002_000.png              No Finding            0           2   
4       00000003_001.png                  Hernia            0           3   
...                  ...                     ...          ...         ...   
112115  00030801_001.png          Mass|Pneumonia            1       30801   
112116  00030802_000.png              No Finding            0       30802   
112117  00030803_000.png              No Finding            0       30803   
112118  00030804_000.png              No Finding            0       30804   
112119  00030805_000.png              No Finding            0       30805   

        Patient Age Patient Gender View Position  OriginalImage[Width  \
0 

In [4]:
individual_lebels = []

all_labels = data_sheet['Finding Labels']

for i in range(N):
    label = all_labels[i] 
    if '|' in label:
        pass
    else:
        individual_lebels.append(label)

print(np.unique(individual_lebels))

['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


In [5]:
g = np.where(np.array(['Atelectasis', 'Cardiomegaly', 'Effusion'])=='Cardiomegaly')[0]
print(g)
print(len(g))

[1]
1


In [6]:
filenames = {}
for disease in disease_names:
    filenames[disease] = []
    
print(filenames)

{'Atelectasis': [], 'Cardiomegaly': [], 'Effusion': [], 'Infiltration': [], 'Mass': [], 'Nodule': [], 'Pneumonia': [], 'Pneumothorax': [], 'Consolidation': [], 'Edema': [], 'Emphysema': [], 'Fibrosis': [], 'Pleural_Thickening': [], 'Hernia': [], 'No Finding': []}


In [21]:
all_files = data_sheet['Image Index']
all_labels = data_sheet['Finding Labels']

full_labels = np.zeros((N,len(disease_names)),dtype=np.float32)

for i in range(N):
    if i%10000 == 0:
        print('Completed ', i, ' of ', N)
    label = all_labels[i]
    for j in range(len(disease_names)):
        disease = disease_names[j]
        
        if disease in label:
            full_labels[i,j] = 1.0
                

Completed  0  of  112120
Completed  10000  of  112120
Completed  20000  of  112120
Completed  30000  of  112120
Completed  40000  of  112120
Completed  50000  of  112120
Completed  60000  of  112120
Completed  70000  of  112120
Completed  80000  of  112120
Completed  90000  of  112120
Completed  100000  of  112120
Completed  110000  of  112120


In [11]:
from imageio import imread 
import PIL 
from PIL import Image 
from torchvision import transforms 

import torch
from torchvision import transforms

device = torch.device("cuda:1") # if torch.cuda.is_available() else "cpu")

preprocess = transforms.Compose([
    transforms.Resize(256),
    #transforms.CenterCrop(224),
    #transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


#all_files = data_sheet['Image Index']
#full_labels

loaded_images= []

for i in range(len(all_files)):

    file_to_load = '/home/user/Chest_X_Ray_Data/CXR14/CXR8/images/extracted/'+all_files[i]
    img = imread(file_to_load, as_gray=False, pilmode="RGB") 
    img = Image.fromarray(img)
    img = PIL.ImageOps.equalize(img)
    img = preprocess(img)
    loaded_images.append(img)


In [14]:
print(len(loaded_images))
print(loaded_images[0:2])

112120
[<PIL.Image.Image image mode=RGB size=256x256 at 0x7F45FF557A90>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F45FF5579E8>]


In [19]:
augments = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def sample_images(images,full_labels, device, n_sample=5):
    N = len(images)
    chs = np.random.choice(N,n_sample)
    augmented_images = []
    for idx in chs:
        img = augments(images[idx])
        augmented_images.append(img)
    
    labels = torch.as_tensor(full_labels[chs]).to(device)
    augmented_images = torch.stack(augmented_images).to(device)
    
    return augmented_images, labels

G, label = sample_images(loaded_images,full_labels,device,n_sample=30)
print(G.size())
print(label)

torch.Size([30, 3, 224, 224])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
model = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=False)
model.classifier = nn.Linear(1024, 15)
model.to(device)
print(model)

import torch.optim as optim

m = nn.Sigmoid()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(10000):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = sample_images(loaded_images,full_labels,device,n_sample=30)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = m(model(inputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.211
[1,  4000] loss: 0.205
[1,  6000] loss: 0.203
[1,  8000] loss: 0.200
[1, 10000] loss: 0.198
[2,  2000] loss: 0.196
[2,  4000] loss: 0.195
[2,  6000] loss: 0.193
[2,  8000] loss: 0.192
[2, 10000] loss: 0.191
[3,  2000] loss: 0.190
[3,  4000] loss: 0.189
[3,  6000] loss: 0.188
[3,  8000] loss: 0.186
[3, 10000] loss: 0.187
[4,  2000] loss: 0.185
[4,  4000] loss: 0.185
[4,  6000] loss: 0.185
[4,  8000] loss: 0.183
[4, 10000] loss: 0.183
[5,  2000] loss: 0.183
[5,  4000] loss: 0.182
[5,  6000] loss: 0.182
[5,  8000] loss: 0.181
[5, 10000] loss: 0.181
[6,  2000] loss: 0.178
[6,  4000] loss: 0.179
[6,  6000] loss: 0.178
[6,  8000] loss: 0.178
[6, 10000] loss: 0.178
[7,  2000] loss: 0.179
[7,  4000] loss: 0.178
[7,  6000] loss: 0.177
[7,  8000] loss: 0.176
[7, 10000] loss: 0.177
[8,  2000] loss: 0.177
[8,  4000] loss: 0.176
[8,  6000] loss: 0.175
[8,  8000] loss: 0.175
[8, 10000] loss: 0.172
[9,  2000] loss: 0.172
[9,  4000] loss: 0.174
[9,  6000] loss: 0.174
[9,  8000] 

In [18]:
torch.save(model.state_dict(), 'saved_model/cxr14_overlapping_labels_bceloss_densenet_121.pth')

In [ ]:
model2 = torch.hub.load('pytorch/vision:v0.5.0', 'densenet121', pretrained=False)
model2.classifier = nn.Linear(1024, 15)

model2.load_state_dict(torch.load('saved_model/cxr14_overlapping_labels_bceloss_densenet_121.pth'))